In [1]:
# import libraries
import PyPDF2
from urllib.request import urlopen
import requests
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import itertools
import json

from pubchemScrapper import hasExperimentalProperties, getFlashpoint, getCompoundName, getSmiles
from pubchemScrapper import strToNegInt, findUnit, findTemp, farnenheitToCelsius
import util

In [2]:
integrated_data = pd.read_csv('../data/integrated_dataset.csv')
print(integrated_data['source'].unique())
integrated_data.isnull().sum()

['pubchem' 'carroll11' 'chen14' 'Saldana11' 'gelest_germanium'
 'gelest_metal-organics' 'pan12' 'gelest_silanes' 'gelest_tin'
 'hazardous materials' 'le15' 'wang11' 'didier14' 'pan07' 'caroll15'
 'didier12']


smiles            0
compound          0
flashpoint        0
source            0
pure substance    0
dtype: int64

In [3]:
integrated_data[integrated_data['smiles'] == 'C1=CC=C(C=C1)[Si](Cl)(Cl)[Cl-]Cl']

,smiles,compound,flashpoint,source,pure substance


# Query pubchem for values with flashpoint

In [3]:
file = open('cid_flashpoint_list.txt', 'r')
df = pd.DataFrame(columns=['cid', 'compound', 'smiles', 'flashpoints']) # make dataframe

In [4]:
line = file.readline()
while line:
    cid = line.strip()
    link = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/'+cid+'/JSON/?response_type=display'

    r = requests.get(link)
    query = r.json()
    #print(cid)
    experimentalProperties = hasExperimentalProperties(query)
    if experimentalProperties != None:
        flashpoints = getFlashpoint(experimentalProperties)
        if flashpoints != None:
            # find compound smiles add to dataframe
            print(cid)
            compound = getCompoundName(query)
            smiles = getSmiles(query)
            newDf = pd.DataFrame
            #print(str(cid)+" "+str(smiles))
            #print(flashpoints)
            df = df.append({'cid #' : cid, 
                            'compound': compound, 
                            'smiles': smiles,
                            'flashpoints': flashpoints}, ignore_index=True)
    line = file.readline()


4
6
11
13
33
34
174
176
177
179
180
196
222
227
240
241
242
243
244
261
262
263
264
273
280
281
284
289
297
299
300
307
308
311
323
335
338
342
356
379
398
402
403
428
454
460
527
533
612
650
660
674
679
700
702
712
713
727
753
768
783
785
787
795
875
878
887
899
904
931
936
938
942
949
957
965
980
991
995
996
999
1017
1030
1031
1032
1049
1068
1110
1127
1133
1140
1146
1183
1206
1254
1322
1567
1727
1732
1752
1988
2078
2087
2244
2263
2266
2286
2345
2347
2365
2450
2537
2710
2730
2758
2871
2879
2907
2955
2969
3007
3017
3026
3030
3031
3036
3039
3048
3080
3082
3102
3118
3121
3283
3286
3289
3293
3301
3314
3346
3438
3485
3598
3767
3776
3893
4004
4093
4101
4116
4130
4133
4156
4169
4284
4510
4618
4632
4650
4684
4685
4735
4763
4790
4792
4793
4837
4932
4933
4936
4944
4947
5002
5054
5126
5144
5192
5251
5281
5392
5447
5455
5497
5541
5543
5561
5565
5569
5641
5773
5780
5794
5801
5921
5942
5971
5976
5993
6001
6010
6028
6050
6054
6061
6101
6108
6113
6115
6124
6129
6163
6173
6184
6212
6227
6228
6251
6264

24870
24874
24946
24998
25006
25054
25305
25353
25418
25457
25485
25508
25511
25515
25608
25644
25670
25750
25914
26176
26177
26318
26323
26374
27054
27582
27756
27882
27909
27924
28500
28594
28700
28777
28803
28838
29740
29935
31200
31211
31215
31217
31218
31229
31234
31236
31237
31238
31241
31242
31244
31245
31246
31247
31249
31253
31254
31256
31260
31261
31263
31264
31268
31271
31272
31275
31276
31278
31279
31285
31287
31288
31289
31296
31297
31298
31344
31347
31356
31357
31368
31369
31370
31373
31374
31404
31405
31412
31423
31621
31677
31957
32485
32490
32611
32739
32791
32803
32881
32898
32965
33023
33097
33101
33112
33360
33363
33510
33528
33557
33599
33600
33744
33934
34023
34148
34692
34697
34763
34766
36392
36400
36442
36565
36679
37175
37517
38018
38037
38103
38479
38759
38764
39149
39385
39793
39985
40470
40818
41478
45056
48132
51132
51605
54744
54778
54901
60835
60854
60987
60994
60999
61009
61011
61020
61021
61061
61103
61136
61140
61195
61246
61247
61249
61250
61258
6128

In [5]:
file.close()
df.head(5)
df.drop(axis = 1, labels='cid', inplace=True)

In [6]:
df.to_csv('pubchem_flashpoints_all.csv', index=False)

In [24]:
#df = pd.read_csv('pubchem_flashpoints_all.csv')

In [20]:
def findUnitv2(data):
    array = data[0]['Value']['StringWithMarkup'][0]['String'].split()
    for value in array:     
        if value == 'F':
            return 'F'
        if value == 'C':
            return 'C'
        if value == 'Â°C':
            return 'C'
        
def tempWithDegreeNoSpace(data):
    if bool(re.search(r'\d', data)):
        if "C" in data or "F" in data:
            return True
    return False
def findTempv2(data):
    try: 
        print(data[0]['Value']['StringWithMarkup'][0]['String'])
        string = data[0]['Value']['StringWithMarkup'][0]['String']#.replace('>', "")
        #string =  string.replace('<', "")
        array = string.split() #data[0]['Value']['StringWithMarkup'][0]['String'].split()
        if array[0] == 'Not' or array[0] == 'Flammable' or array[0] == '(Non-Specific' or array[0] == 'No' or array[0] == 'Explodes' or array[0] == 'see':
            return "no fp", "no unit"
        if array[0] == 'High' or array[0] == 'None' or array[0] == 'Non-flammable' or array[0] == 'none':
            return "no fp", "no unit"
        if array[0] == '(explodes)' or array[0] == '/Calcium' or array[0] == 'NA' or array[0] == 'Non-combustibile':
            return "no fp", "no unit"
        if '>' in string or '<' in string or 'greater than' in string or 'less than' in string:
            return "no fp", "no unit"
        if 'ABATE ' in string or '~' in string or 'Less than' in string or 'GREATER THAN' in string or 'BELOW' in string or 'Above' in string or 'Greater' in string or 'commercial' in string:
            return "no fp", "no unit"
        if 'ABOUT' in string or 'Tordon' in string or 'APPROX' in string or 'Approximately' in string:
            return "no fp", "no unit"
        
        for value in array:
            # check for range then take min temp
            if re.search('\\w+-\\w', value) != None:
                index = value.find('-')
                value = value[0:index]
                return "no fp", "no unit"
            #check if value has a decimal
            if value.find('.') != -1:
                index = value.find('.')
                value = value[0:index]
            if tempWithDegreeNoSpace(value) == True:
                if "C" in value or "F" in value:
                    if value.find('F') != -1:
                        index = value.find('F')
                        value = value[0:index]
                        if value.find('Â°') != -1:
                            index = value.find('Â°')
                            value = value[0:index]
                        if value[0] == '-':
                            value = strToNegInt(value)
                        return int(value), "F"
                    if value.find('C') != -1:
                        index = value.find('C')
                        value = value[0:index]
                        if value.find('Â°') != -1:
                            index = value.find('Â°')
                            value = value[0:index] 
                        if value[0] == '-':
                            value = strToNegInt(value)
                        return int(value), "C" 
            if value == '-':
                # ugh stupid edge case
                temp = int(array[1]) * -1
                return temp, "callfindUnit"
            if value[0] == '-':
                temp = strToNegInt(value)
                return temp, "callfindUnit"
            if value.isdigit() == True:
                temp = int(value)
                return temp, "callfindUnit"
                        
        return temp, "callfindUnit"
    
    except UnboundLocalError:
        print(string)
    return "no fp", "no unit"

def farnenheitToCelsiusv2(temp):
    return round((temp - 32) * (5 / 9))

def celsiusToKelvin(temp):
    return temp + 273.15

In [21]:
df['flashpoint'] = None
df['unit'] = None
df.head(5)

,compound,smiles,flashpoints,cid #,flashpoint,unit
0,1-aminopropan-2-ol,CC(CN)O,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',...",4,None,None
1,"1-chloro-2,4-dinitrobenzene",C1=CC(=C(C=C1[N+](=O)[O-])[N+](=O)[O-])Cl,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',...",6,None,None
2,"1,2-dichloroethane",C(CCl)Cl,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',...",11,None,None
3,"1,2,4-trichlorobenzene",C1=CC(=C(C=C1Cl)Cl)Cl,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',...",13,None,None
4,2-chloroacetaldehyde,C(C=O)Cl,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',...",33,None,None


In [22]:
for index, row in df.iterrows():
    #print(row['cid #'])
    temp, unit = findTempv2(row.flashpoints)
    #print(unit)
    if unit == 'callfindUnit':
        unit = findUnitv2(row.flashpoints)
    #if unit == 'F':
    #    temp = farnenheitToCelsius(temp)
    row['flashpoint'] = temp
    row['unit'] = unit
    if temp == 'no fp':
        print('no fp value')
    else:
        if unit is None:
            print(str(temp) + ' ' + 'no unit')
        else:
            print(str(temp) + ' ' + unit)

df.head(5)

171 Â° F (NTP, 1992)
171 F
382 Â° F (NFPA, 2010)
382 F
56 Â° F (NTP, 1992)
56 F
230 Â° F (NTP, 1992)
230 F
190 Â° F (NTP, 1992)
190 F
140 Â° F (EPA, 1998)
140 F
232 Â° F (NTP, 1992)
232 F
104 Â° F (NTP, 1992)
104 F
-40 Â° F (NTP, 1992)
-40 F
106 deg F
106 F
0 Â° F (NTP, 1992)
0 F
376 Â° F Combustible solid (USCG, 1999)
376 F
132 deg C (270 deg F) - closed cup
132 C
340 Â° F (NTP, 1992)
340 F
148 Â° F (NTP, 1992)
148 F
12 Â° F (NTP, 1992)
12 F
250 Deg F (121 Deg C) (closed cup)
250 F
250 Â° F (NTP, 1992)
250 F
213 Â° F (NTP, 1992)
213 F
20 Â° F (NTP, 1992)
20 F
85 deg C (185 deg F) - closed cup
85 C
84 Â° F (USCG, 1999)
84 F
170 Â° F (NTP, 1992)
170 F
62 deg C (144 deg F) - closed cup
62 C
Not applicable
no fp value
Flammable gas
no fp value
156 Â° F (NTP, 1992)
156 F
261 Â° F (NTP, 1992)
261 F
-306 Â° F (NTP, 1992)
-306 F
>100.00 deg C (>212.00 deg F)
no fp value
302 Â° F (EPA, 1998)
302 F
120 deg C closed cup
120 C
126 DEG C
126 C
100 Â°C
100 C
304 Â° F (NTP, 1992)
304 F
177.8 to 181.

124 deg C (255 deg F) - closed cup
124 C
270 Â° F for a 80% 2,4:20% 2,6 TDI mixture (EPA, 1998)
270 F
162 Â°C
162 C
134 Â° F (USCG, 1999)
134 F
138 Â° F (NFPA, 2010)
138 F
142 deg C closed cup
142 C
286 DEG F CC
286 F
208 Â° F (NTP, 1992)
208 F
125 Â°C
125 C
315 Â° F (NIOSH, 2016)
315 F
greater than 235 Â° F (NTP, 1992)
no fp value
175 Â° F (NTP, 1992)
175 F
185 Â° F (NTP, 1992)
185 F
>200 Â°C
no fp value
375 Â° F (NIOSH, 2016)
375 F
266 Â° F (NTP, 1992)
266 F
235 Â° F (NTP, 1992)
235 F
greater than 200 Â° F (NTP, 1992)
no fp value
290 Â° F (NTP, 1992)
290 F
greater than 235 Â° F (NTP, 1992)
no fp value
330 DEG F
330 F
>150 Â°C c.c.
no fp value
202 Â°C
202 C
290 Â° F (NIOSH, 2016)
290 F
405 Â° F (NIOSH, 2016)
405 F
greater than 230 Â° F (NTP, 1992)
no fp value
210 Â° F (NTP, 1992)
210 F
181 Â° F (NTP, 1992)
181 F
186 DEG C OC; 15 DEG C CC
186 C
265-420 DEG F (OPEN CUP) /2,4,5-T ESTERS/
no fp value
Greater than 175 deg F (open cup) /2,4-D esters/
no fp value
181 deg C
181 C
176 Â° F (NI

106 F
30 Â° F (NTP, 1992)
30 F
-18 Â° F (USCG, 1999)
-18 F
36 Â° F (NTP, 1992)
36 F
60 Â° F (NFPA, 2010)
60 F
60.1 Â° F (EPA, 1998)
60 F
121 Â° F (NTP, 1992)
121 F
178 Â° F (NTP, 1992)
178 F
157 Â°C
157 C
218 Â° F (NTP, 1992)
218 F
116 deg C (241 deg F) - closed cup
116 C
85 Â° F (NTP, 1992)
85 F
126 Â° F (NTP, 1992)
126 F
255 Â° F (NTP, 1992)
255 F
greater than 233.6 Â° F (NTP, 1992)
no fp value
186 Â° F (USCG, 1999)
186 F
280 Â° F (NTP, 1992)
280 F
185 Â° F (NTP, 1992)
185 F
114 Â° F (USCG, 1999)
114 F
122 Â° F (NIOSH, 2016)
122 F
80 Â°C c.c.
80 C
200 Â° F (NTP, 1992)
200 F
224.6 Â° F (NTP, 1992)
224 F
136 DEG F
136 F
165 Â° F (USCG, 1999)
165 F
140 Â° F (NTP, 1992)
140 F
113 Â° F (USCG, 1999)
113 F
123.8 Â° F (NTP, 1992)
123 F
25 Â° F (USCG, 1999)
25 F
134 Â° F (NTP, 1992)
134 F
75 Â° F (NTP, 1992)
75 F
88 Â° F (EPA, 1998)
88 F
154 Â° F (NTP, 1992)
154 F
111 Â° F (NTP, 1992)
111 F
127 Â° F (USCG, 1999)
127 F
97 Â° F (NTP, 1992)
97 F
75 Â° F (NFPA, 2010)
75 F
97 Â° F (NTP, 1992)
97 F

221 F
Not Applicable. Not flammable. (USCG, 1999)
no fp value
Not Applicable. Not flammable. (USCG, 1999)
no fp value
75 Â°C
75 C
Flash point: -16.3 deg C
-16 C
150 deg F (60 deg C) /closed cup/
150 F
greater than 235 Â° F (NTP, 1992)
no fp value
see Notes
no fp value
greater than 300 Â° F (NTP, 1992)
no fp value
less than 20 Â° F (NFPA, 2010)
no fp value
-10 Â°C
-10 C
30 Â° F (NTP, 1992)
30 F
92 Â° F (USCG, 1999)
92 F
-10 deg F (-23 deg C) (Closed cup)
-10 F
187 Â°C
187 C
greater than 230 Â° F (NTP, 1992)
no fp value
111 deg F (44 deg C) (Closed cup)
111 F
61-95 Â°C c.c.
no fp value
146 Â° F (USCG, 1999)
146 F
302 Â° F (USCG, 1999)
302 F
184 Â° F (NTP, 1992)
184 F
153 Â° F (NTP, 1992)
153 F
265 DEG F
265 F
244 Â° F (NTP, 1992)
244 F
179.5 Â° F (NTP, 1992)
179 F
93 deg C
93 C
BELOW 80 DEG F
no fp value
-22 Â° F (NTP, 1992)
-22 F
None (EPA, 1998)
no fp value
greater than 235 Â° F (NTP, 1992)
no fp value
9.7 deg C (49.5 deg F) - closed cup
9 C
High enough not to interfere with military u

64 F
66 DEG F OC
66 F
95 Â° F (USCG, 1999)
95 F
102 deg F, 39 deg C (TCC)
102 C
75.2 Â° F (NTP, 1992)
75 F
115 Â° F (NTP, 1992)
115 F
147 Â° F (NIOSH, 2016)
147 F
4 deg C (39 deg F) - closed cup
4 C
greater than 200 Â° F (NTP, 1992)
no fp value
Depending on the specific mixture, flash points may vary from 30-141 deg F. /Motor fuel anti-knock mixtures; Motor fuel anti-knock compounds/
no fp value
113 Â°C c.c.
113 C
greater than 200 Â° F (NTP, 1992)
no fp value
>200 Â°C c.c.
no fp value
300 Â° F (NTP, 1992)
300 F
69 Â°C c.c.
69 C
Tordon 22k: 46 deg c toc; tordon 101 mixture--combustible with an toc flash point of 35 deg c; Tordon 155 mixture--104 deg c coc /Tordon/
no fp value
>110 deg C (>230 deg F) - closed cup
no fp value
Greater than 175 deg F (open cup) /2,4-D esters/
no fp value
Greater than 175 deg F (open cup) /2,4-D esters/
no fp value
252 DEG F 0C /TECHNICAL/
252 F
100 Â°C c.c.
100 C
>115 Â°C o.c.
no fp value
340 Â° F (NTP, 1992)
340 F
greater than 191 Â° F (NTP, 1992)
no fp va

100.00 deg C (212.00 deg F) - closed cup
100 C
122 deg C (closed cup) /from table/
122 C
85 Â°C o.c.
85 C
350 Â°C
350 C
385 deg F, 196 deg C (closed cup)
385 C
greater than 200 Â° F (NTP, 1992)
no fp value
>70.00 deg C (>158.00 deg F)
no fp value
> 150 deg C (> 302 deg F)
no fp value
9.7 deg C (49.5 deg F) - closed cup
9 C
82 DEG C ; 180 DEG F /TECHNICAL PRODUCT/
82 C
171 Â°C
171 C
Approximately 150F or higher (EPA, 1998)
no fp value
118 DEG C OC
118 C
115 Â° F (USCG, 1999)
115 F
190 deg F (88 deg C) /closed cup/
190 F
152-154 deg C; Cleveland open cup
no fp value
52 Â°C
52 C
108 Â°C
108 C
95 Â° F (NFPA, 2010)
95 F
36 Â° F (NTP, 1992)
36 F
196 Â° F (NTP, 1992)
196 F
150 deg F (60 deg C) /closed cup/
150 F
119 Â° F (NTP, 1992)
119 F
Flash point : 88 deg F
88 F
91 DEG F (33 DEG C) (CLOSED CUP)
91 F
190 deg F (88 deg C) /closed cup/
190 F
190 deg F (88 deg C) /closed cup/
190 F
150 Â° F (NTP, 1992)
150 F
273 deg C (open cup)
273 C
372 Â° F (NTP, 1992)
372 F
55 Â° F (EPA, 1998)
55 F
104 Â°

320 deg F (min), open cup
320 F
530 DEG F Open Cup
530 F
74 Â°C
74 C
315 Â° F (40% hydrogenated) (NIOSH, 2016)
315 F
37.8 Â°C c.c.
37 C
26 deg C
26 C
12 deg F (-11 deg C) (Closed cup)
12 F
greater than 200 Â° F (NTP, 1992)
no fp value
greater than 200 Â° F (NTP, 1992)
no fp value
11 Â°C c.c.
11 C
140 DEG F CC
140 F
greater than 200 Â° F (NTP, 1992)
no fp value
140 DEG F CC
140 F
187 Â°C
187 C
> 113 degrees C (> 235 degrees F) - closed cup
no fp value
GREATER THAN 212 DEG F CC
no fp value
Approximately 150F or higher (EPA, 1998)
no fp value
Solution: 225 deg F (open cup), 132 deg F (closed cup)
225 F
39 Â° F (8-10% solution in toluene) (NTP, 1992)
39 F
GREATER THAN 212 DEG F CC
no fp value
177 deg C (351 deg F) - closed cup
177 C
-8 Â°C c.c.
-8 C
170 Â° F (USCG, 1999)
170 F
170 Â° F (USCG, 1999)
170 F
165 deg C (329 deg F) - closed cup
165 C
65 Â°C
65 C
Flash point is> 100 deg C,
no fp value
-49 Â° F (ether) (USCG, 1999)
-49 F
140 DEG F CC
140 F
205 Â°C
205 C
53 deg F 12 deg C Method: c

,compound,smiles,flashpoints,cid #,flashpoint,unit
0,1-aminopropan-2-ol,CC(CN)O,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',...",4,171,F
1,"1-chloro-2,4-dinitrobenzene",C1=CC(=C(C=C1[N+](=O)[O-])[N+](=O)[O-])Cl,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',...",6,382,F
2,"1,2-dichloroethane",C(CCl)Cl,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',...",11,56,F
3,"1,2,4-trichlorobenzene",C1=CC(=C(C=C1Cl)Cl)Cl,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',...",13,230,F
4,2-chloroacetaldehyde,C(C=O)Cl,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',...",33,190,F


In [46]:
df.head(5)
#df.to_csv('pubchem_flashpoints_all.csv', index=False)

,compound,smiles,flashpoints,cid #,flashpoint,unit
0,1-aminopropan-2-ol,CC(CN)O,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',...",4,171,F
1,"1-chloro-2,4-dinitrobenzene",C1=CC(=C(C=C1[N+](=O)[O-])[N+](=O)[O-])Cl,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',...",6,382,F
2,"1,2-dichloroethane",C(CCl)Cl,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',...",11,56,F
3,"1,2,4-trichlorobenzene",C1=CC(=C(C=C1Cl)Cl)Cl,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',...",13,230,F
4,2-chloroacetaldehyde,C(C=O)Cl,"[{'ReferenceNumber': 1, 'Name': 'Flash Point',...",33,190,F


In [24]:
copy = df

In [25]:
print(copy.shape)
df = df.loc[df['flashpoint'] != "no fp"]
df = df.loc[df['unit'] != "no unit"]
print(df.shape)

(2197, 6)
(1756, 6)


In [55]:
df.loc[df['unit'] == "C"].shape
df.loc[df['unit'] == "F"].shape

(1596, 6)

In [26]:
for index, row in df.iterrows():
    if row['unit'] == 'F':
        row['flashpoint'] = farnenheitToCelsiusv2(row['flashpoint'])
        row['unit'] = 'C'

In [28]:
for index, row in df.iterrows():
        row['flashpoint'] = util.celsiusToKelvin(row['flashpoint'])
        row['unit'] = 'K'
#celsiusToKelvin

In [32]:
result = df.drop(columns=['flashpoints', 'cid #', 'unit'])
result['source'] = 'pubchem' 
result['pure substance'] = 1
#result.to_csv('pubchem_final.csv', index=False)
#result.head()
result.head(5)

,compound,smiles,flashpoint,source,pure substance
0,1-aminopropan-2-ol,CC(CN)O,350.15,pubchem,1
1,"1-chloro-2,4-dinitrobenzene",C1=CC(=C(C=C1[N+](=O)[O-])[N+](=O)[O-])Cl,467.15,pubchem,1
2,"1,2-dichloroethane",C(CCl)Cl,286.15,pubchem,1
3,"1,2,4-trichlorobenzene",C1=CC(=C(C=C1Cl)Cl)Cl,383.15,pubchem,1
4,2-chloroacetaldehyde,C(C=O)Cl,361.15,pubchem,1


In [34]:
result.dropna(inplace=True)
print(result.shape)
result.to_csv('../data/pubchem_data.csv', index=False)


(1756, 5)


# Testing and debugging

In [124]:
value = '150F'
if tempWithDegreeNoSpace(value) == True:
    if "C" in value or "F" in value:
        if value.find('F') != -1:
            index = value.find('F')
            value = value[0:index]
            print(value)
        if value.find('C') != -1:
            index = value.find('C')
            value = value[0:index]

150


In [8]:
data = pd.read_csv('Carroll_Chen_Pan_Saldana_Gelest_merge_1.csv')
pubchem = pd.read_csv('pubchem_final.csv')

In [10]:
data.drop(columns=['Unnamed: 0', 'source'], inplace=True)
data.rename(columns={'data': 'flashpoint'}, inplace=True)
data['smiles'] = data['name'].apply(util.getSmiles)

In [11]:
pubchem['flashpoint'] = pubchem['flashpoint'].apply(celsiusToKelvin)

In [23]:
data.drop(columns=['name'], inplace=True)

In [26]:
data = data[data.columns[::-1]]
data.head(5)
#data.drop_duplicates('smiles', inplace=True).shape

,smiles,flashpoint
0,CCCCC,233.0
1,CCCCCC,247.0
2,CCCCCCC,269.0
3,CCCCCCCC,286.0
4,CCCCCCCCC,304.0


In [27]:
result = pd.concat([pubchem, data])
#pubchem.drop_duplicates('smiles', inplace=True).shape

In [29]:
result.shape

(4435, 2)

In [30]:
result.drop_duplicates('smiles', inplace=True)
result.shape

(3956, 2)

In [32]:
result.to_csv('flashpoint_data.csv', index=False)

In [35]:
print('mean : ' + str(result.flashpoint.mean()))
print('std dev : ' + str(result.flashpoint.std()))

mean : 343.4680017694641
std dev : 67.4267854377623
